In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
# pio.renderers.default = "default"
pio.renderers.default = "vscode"


from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from sklearn.preprocessing import MinMaxScaler


import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
database = pd.read_csv(r"C:\Users\Anson\OneDrive\Anson File\PythonApplication_anson\3_Kaggle\4_20210419_StockPrice_Prediction_LSTM\all_NVDA_1.csv", sep=',')
# database = pd.read_csv(r"C:\Users\lo.anson\OneDrive\Anson File\PythonApplication_anson\3_Kaggle\4_20210419_StockPrice_Prediction_LSTM\all_NVDA_1.csv", sep=',')


database['Date/Time']=pd.to_datetime(database['Date/Time'])
database.set_axis(database['Date/Time'], inplace=True)
stock_data_original=database[["Ticker","Open","High","Low","Volume","Close"]]



###Extraction of all features and targeted closing value for NVIDIA 

stock_data_original_NVDA= stock_data_original[stock_data_original["Ticker"] == "NVDA"]
stock_data_original_NVDA= stock_data_original_NVDA.drop(columns=['Ticker'])

#Target Close Value
stock_data_original_y= stock_data_original_NVDA[["Close"]]
stock_data_original_y.rename(columns={stock_data_original_y.columns[0]: "Close Target"},inplace = True)


###Extraction of closing value for UUP
stock_data_original_UUP= stock_data_original[stock_data_original["Ticker"] == "UUP"]
stock_data_original_UUP= stock_data_original_UUP[["Close","Volume"]]
stock_data_original_UUP.rename(columns={stock_data_original_UUP.columns[0]: "UUP Close",stock_data_original_UUP.columns[1]: "UUP Volume"},inplace = True)

###Extraction of closing value for GDX
stock_data_original_GDX= stock_data_original[stock_data_original["Ticker"] == "GDX"]
stock_data_original_GDX= stock_data_original_GDX[["Close","Volume"]]
stock_data_original_GDX.rename(columns={stock_data_original_GDX.columns[0]: "GDX Close" ,stock_data_original_GDX.columns[1]: "GDX Volume"},inplace = True)


###Extraction of closing value for QQQ
stock_data_original_QQQ= stock_data_original[stock_data_original["Ticker"] == "QQQ"]
stock_data_original_QQQ= stock_data_original_QQQ[["Close","Volume"]]
stock_data_original_QQQ.rename(columns={stock_data_original_QQQ.columns[0]: "QQQ Close" ,stock_data_original_QQQ.columns[1]: "QQQ Volume"},inplace = True)



####Concatenate 
dfs = [stock_data_original_NVDA,stock_data_original_UUP,stock_data_original_GDX,stock_data_original_QQQ,stock_data_original_y]
stock_data_original_merged = pd.concat(dfs, join='outer', axis=1)


print('stock_data_original_merged_ \n\n',stock_data_original_merged)



####Simple Data Cleaning to remove the NaN values in the dataframe
stock_data_original_merged.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
stock_data_values=stock_data_original_merged.values

print('stock_data_values \n\n',stock_data_values)

#IMPORTANT
# print('database[Date/Time] \n\n',stock_data_original_merged)
# print('datastock_data \n\n',stock_data_values)
# print('stock_data_values.iloc[:,0:10]',stock_data_values[:,0:10])
# print('stock_data_values.iloc[:,10]',stock_data_values[:,10])

stock_data_original_merged_ 

               Open    High     Low      Volume   Close  UUP Close  UUP Volume  \
Date/Time                                                                       
2007-03-01   20.22   20.79   19.95  20930700.0   20.41      24.97      4600.0   
2007-03-02   20.23   20.48   19.81  17068200.0   19.85      24.96      2400.0   
2007-03-05   19.50   19.89   19.11  16588900.0   19.17      25.12      1100.0   
2007-03-06   19.59   20.53   19.51  20149300.0   20.31      25.10       400.0   
2007-03-07   20.21   20.30   19.65  20291400.0   19.73      25.00      2800.0   
...            ...     ...     ...         ...     ...        ...         ...   
2021-07-02  817.61  820.21  811.51   8569100.0  819.48      24.80   1107400.0   
2021-07-06  829.50  833.68  814.01  11167700.0  827.94      24.89   1152100.0   
2021-07-07  834.14  835.00  813.28  10460700.0  814.87      24.93    699100.0   
2021-07-08  794.26  805.32  788.03  12582500.0  796.11      24.83   1236800.0 

In [3]:
# Divide the data for training and testing
# 80% for training data, 30% for testing data

split_percent = 0.70
split = int(split_percent*len(stock_data_original_NVDA))


In [4]:

stock_data_X = stock_data_values[:,0:stock_data_values.shape[1]-1]
stock_data_y = stock_data_values[:,stock_data_values.shape[1]-1]


# Normalization for stock_data for neural network

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
stock_data_X_scaled = scaler_X.fit_transform(stock_data_X)
stock_data_y_scaled = scaler_y.fit_transform(stock_data_y.reshape(-1,1)).reshape(-1)


stock_data_train_X = stock_data_X_scaled[:split]
stock_data_train_y = stock_data_y_scaled[:split]

stock_data_test_X = stock_data_X_scaled[split:]
stock_data_test_y = stock_data_y_scaled[split:]

print('split',split)

# Split for datatime 


date_train = database['Date/Time'][:split]
date_test = database['Date/Time'][split:len(stock_data_original_NVDA)-6]


# print('date_train',date_train)
# print('date_test',date_test)

split 2535


In [5]:
#look back day is 50
look_back = 50

train_generator = TimeseriesGenerator(data=stock_data_train_X, targets=stock_data_train_y, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(data=stock_data_test_X, targets=stock_data_test_y, length=look_back, batch_size=1)



# print("stock_data_test_X")
# for p in stock_data_test_X: 
#   print (p)

# print("stock_data_test_y")
# for p in stock_data_test_y: 
#   print (p)


In [6]:
Keras_model = Sequential()
Keras_model.add(LSTM(100, activation='relu',input_shape=(look_back,stock_data_train_X.shape[1])))
Keras_model.add(Dropout(0.3))
Keras_model.add(Dense(1))
Keras_model.compile(optimizer='adam',loss='mean_squared_error')

num_epochs = 10
Keras_model.fit_generator(train_generator, epochs=num_epochs, verbose=1)



C:\Users\Anson\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1240: UserWarning:

`model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.

Epoch 1/10
125/125 [==============================] - 20s 140ms/step - batch: 62.0000 - size: 19.8800 - loss: 8.5462e-04
Epoch 2/10
125/125 [==============================] - 18s 143ms/step - batch: 62.0000 - size: 19.8800 - loss: 2.4825e-04
Epoch 3/10
125/125 [==============================] - 19s 155ms/step - batch: 62.0000 - size: 19.8800 - loss: 1.1107e-04
Epoch 4/10
125/125 [==============================] - 19s 155ms/step - batch: 62.0000 - size: 19.8800 - loss: 7.9182e-05
Epoch 5/10
125/125 [==============================] - 19s 153ms/step - batch: 62.0000 - size: 19.8800 - loss: 7.0333e-05
Epoch 6/10
125/125 [==============================] - 20s 156ms/step - batch: 62.0000 - size: 19.8800 - loss: 5.6559e-05
Epoch 7/10
125/125 [================

In [7]:
prediction = Keras_model.predict_generator(test_generator)

# for p in test_generator: 
#   print (p)


C:\Users\Anson\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1301: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.

C:\Users\Anson\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning:

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.



In [8]:
def predict(num_prediction, Keras_model):
    prediction_list_X = stock_data_test_X[-look_back:]
    prediction_list_y = stock_data_test_y[-look_back:]
    


    
    for _ in range(num_prediction):
        x = prediction_list_X[-look_back:]
        x = x.reshape((1, look_back, stock_data_test_X.shape[1]))
        out = Keras_model.predict(x)[0][0]
        prediction_list_y = np.append(prediction_list_y, out)
#         print('prediction_list_y',prediction_list_y)
        
        
    prediction_list_y = prediction_list_y[look_back-1:]

    print('prediction_list_y',prediction_list_y)

    return prediction_list_y

In [9]:
def predict_dates(num_prediction):

    last_date = database['Date/Time'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    
    print('prediction_dates',prediction_dates)
    return prediction_dates


In [10]:
# Prediction the Future Price

num_prediction = 1
forecast = predict(num_prediction, Keras_model)
forecast_dates = predict_dates(num_prediction)

# print('forecast',forecast)
# print('forecast_dates',forecast_dates)

prediction_list_y [0.96845652 0.96167845]
prediction_dates [Timestamp('2021-07-09 00:00:00', freq='D'), Timestamp('2021-07-10 00:00:00', freq='D')]


In [11]:

# print('stock_data_train_y.reshape',stock_data_train_y)
# print('stock_data_train_y.reshape(-1,1)',stock_data_train_y.reshape(-1,1))
# print('stock_data_test_y',stock_data_test_y)


# stock_data_y = scaler_y.inverse_transform(stock_data_y_scaled.reshape(-1,1)).reshape(-1)
# print('stock_data_y',stock_data_y)
# print('stock_data_original_y',stock_data_original_y.values[split:].reshape(-1))

stock_data_train = stock_data_original_y.values.reshape(-1)[:split]
stock_data_test = stock_data_original_y.values.reshape(-1)[split:]

print('prediction',prediction)
# for p in prediction: 
#   print (p)

stock_data_y_test_prediction = scaler_y.inverse_transform(prediction.reshape(-1,1)).reshape(-1)


print('stock_data_y_test_prediction', stock_data_y_test_prediction)
# for p in stock_data_y_test_prediction:
#     print(p)



# print('prediction',prediction.reshape(-1,1))
# print('stock_data_y_test_prediction',stock_data_y_test_prediction)


forecast = scaler_y.inverse_transform(forecast.reshape(-1,1))
forecast = forecast.reshape(-1)


# ##IMPORTANT
# print('stock_data_train',stock_data_train)
# print('stock_data_test',stock_data_test)
# print('forecast',forecast)

# print('date_train',date_train)
# print('date_test',date_test)
# print('forecast_dates',forecast_dates)


prediction [[0.13468914]
 [0.13702026]
 [0.13780734]
 ...
 [0.9619258 ]
 [0.9707131 ]
 [0.9637739 ]]
stock_data_y_test_prediction [116.61986 118.53613 119.18314 ... 796.64154 803.865   798.1607 ]


In [12]:
# Graph for the stock prediction

trace1 = go.Scatter(x = date_train, y = stock_data_train, mode = 'lines', name = 'Data')
trace2 = go.Scatter(x = date_test,y = stock_data_test,mode='lines', name = 'Ground Truth')
trace3 = go.Scatter(x = date_test,y = stock_data_y_test_prediction ,mode='lines', name = 'Test Prediction')
trace4 = go.Scatter(x = forecast_dates,y = forecast, mode = 'lines', name = 'Future')
layout = go.Layout(title = "Nvidia Stock", xaxis = {'title' : "Date"}, yaxis = {'title' : "Close"})
fig = go.Figure(data=[trace1,trace2,trace3,trace4], layout=layout)
fig.show()